## Movie Recommendation System : ระบบแนะนำหนัง

## Content-Based Filtering

In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [2]:
df1 = pd.read_csv('tmdb_5000_credits.csv.zip')
df2 = pd.read_csv('tmdb_5000_movies.csv.zip')

In [3]:
df1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
df2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


#### นำตาราง df1 และ  df2 ที่ movie id เหมือนกันมา merge กัน

In [5]:
movie = pd.merge(df1 , df2 , left_on='movie_id' , right_on='id')

In [6]:
movie.head()

,movie_id,title_x,cast,crew,budget,genres,homepage,id,keywords,original_language,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [7]:
movie.columns

Index(['movie_id', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage',
       'id', 'keywords', 'original_language', 'original_title', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title_y', 'vote_average', 'vote_count'],
      dtype='object')

#### เลือก columns ที่ใช้

In [8]:
movie = movie[['title_x','cast', 'crew','genres','keywords','overview']].reset_index()

In [9]:
movie.head()

,index,title_x,cast,crew,genres,keywords,overview
0,0,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,1,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,2,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,3,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,4,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [10]:
movie.rename(columns={'title_x' : 'title'} , inplace=True)

In [11]:
movie.columns

Index(['index', 'title', 'cast', 'crew', 'genres', 'keywords', 'overview'], dtype='object')

#### ทำให้เป็นรูปแบบ json เพื่อเข้าถึงข้อมูล

In [12]:
for i in movie[['cast', 'crew', 'genres', 'keywords']]:
    movie[i] = movie[i].apply(json.loads)

In [13]:
movie.head()

,index,title,cast,crew,genres,keywords,overview
0,0,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","In the 22nd century, a paraplegic Marine is di..."
1,1,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","Captain Barbossa, long believed to be dead, ha..."
2,2,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",A cryptic message from Bond’s past sends him o...
3,3,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",Following the death of District Attorney Harve...
4,4,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...","John Carter is a war-weary, former military ca..."


In [14]:
def lis_name(x):
    lis = []
    for i in x :
        lis.append(i['name'])     
    return lis

In [15]:
def director_name(x):
    lis = []
    for i in x :
        if i['job'] == 'Director':
            lis.append(i['name'])
    return lis

In [16]:
for i in movie[['cast','genres','keywords']]:
    movie[i] = movie[i].apply(lis_name)

In [17]:
movie['crew'] = movie['crew'].apply(director_name)

#### ตาราง movie

In [18]:
movie.head()

,index,title,cast,crew,genres,keywords,overview
0,0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."
2,2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes],"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...
3,3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan],"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...
4,4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton],"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca..."


#### ตรวจสอบ missing value 

In [19]:
movie.isna().sum()

index       0
title       0
cast        0
crew        0
genres      0
keywords    0
overview    3
dtype: int64

In [20]:
movie['overview'] = movie['overview'].fillna('') 

#### overview : เรื่องย่อ

In [21]:
movie['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [22]:
movie['movie_info']  = movie['cast'] + movie['crew'] + movie['genres'] + movie['keywords']

#### movie_info : ข้อมูลหนัง (นักแสดง ผู้กำกับ ประเภทหนัง แนวหนัง)

In [23]:
movie['movie_info'][0]

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Giovanni Ribisi',
 'Joel David Moore',
 'CCH Pounder',
 'Wes Studi',
 'Laz Alonso',
 'Dileep Rao',
 'Matt Gerald',
 'Sean Anthony Moran',
 'Jason Whyte',
 'Scott Lawrence',
 'Kelly Kilgour',
 'James Patrick Pitt',
 'Sean Patrick Murphy',
 'Peter Dillon',
 'Kevin Dorman',
 'Kelson Henderson',
 'David Van Horn',
 'Jacob Tomuri',
 'Michael Blain-Rozgay',
 'Jon Curry',
 'Luke Hawker',
 'Woody Schultz',
 'Peter Mensah',
 'Sonia Yee',
 'Jahnel Curfman',
 'Ilram Choi',
 'Kyla Warren',
 'Lisa Roumain',
 'Debra Wilson',
 'Chris Mala',
 'Taylor Kibby',
 'Jodie Landau',
 'Julie Lamm',
 'Cullen B. Madden',
 'Joseph Brady Madden',
 'Frankie Torres',
 'Austin Wilson',
 'Sara Wilson',
 'Tamica Washington-Miller',
 'Lucy Briant',
 'Nathan Meister',
 'Gerry Blair',
 'Matthew Chamberlain',
 'Paul Yates',
 'Wray Wilson',
 'James Gaylyn',
 'Melvin Leno Clark III',
 'Carvon Futrell',
 'Brandon Jelkes',
 'Mica

#### นำ text มา join กัน

In [24]:
movie['movie_info'] = movie['movie_info'].apply(lambda x : ' '.join(x))

In [25]:
movie['movie_info'][0]

'Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Giovanni Ribisi Joel David Moore CCH Pounder Wes Studi Laz Alonso Dileep Rao Matt Gerald Sean Anthony Moran Jason Whyte Scott Lawrence Kelly Kilgour James Patrick Pitt Sean Patrick Murphy Peter Dillon Kevin Dorman Kelson Henderson David Van Horn Jacob Tomuri Michael Blain-Rozgay Jon Curry Luke Hawker Woody Schultz Peter Mensah Sonia Yee Jahnel Curfman Ilram Choi Kyla Warren Lisa Roumain Debra Wilson Chris Mala Taylor Kibby Jodie Landau Julie Lamm Cullen B. Madden Joseph Brady Madden Frankie Torres Austin Wilson Sara Wilson Tamica Washington-Miller Lucy Briant Nathan Meister Gerry Blair Matthew Chamberlain Paul Yates Wray Wilson James Gaylyn Melvin Leno Clark III Carvon Futrell Brandon Jelkes Micah Moch Hanniyah Muhammad Christopher Nolen Christa Oliver April Marie Thomas Bravita A. Threatt Colin Bleasdale Mike Bodnar Matt Clayton Nicole Dionne Jamie Harrison Allan Henry Anthony Ingruber Ashley Jeffery Dean Kn

In [27]:
tv = TfidfVectorizer(stop_words='english')
cv = CountVectorizer(stop_words='english')

#### Content-Based Filtering โดย movie_info

In [28]:
tv_matrix = tv.fit_transform(movie['movie_info'])

In [29]:
tv_matrix.shape

(4803, 41825)

#### หา movie ใกล้เคียงโดย cosine_similarity (ความใกล้เคียงตามมุม)

In [30]:
cosin_sim = cosine_similarity(tv_matrix)

In [31]:
cosin_sim.shape

(4803, 4803)

#### เลือกหนังมา 1 เรื่อง : Batman

In [32]:
movie[movie['title'] == 'Batman'].index.values[0]

1359

#### Index ของ Batman = 1359

In [33]:
cosin_sim[1359]

array([0.01726709, 0.04950062, 0.03265167, ..., 0.        , 0.        ,
       0.        ])

#### แสดง index และ ค่าความใกล้เคียงของ index 1359

In [34]:
list(enumerate(cosin_sim[1359]))[:10]

[(0, 0.0172670860682558),
 (1, 0.04950061652887749),
 (2, 0.03265167254165795),
 (3, 0.09883465868752744),
 (4, 0.006912474322677083),
 (5, 0.05522429054806459),
 (6, 0.013851573873784858),
 (7, 0.047292298003772604),
 (8, 0.019097303420393712),
 (9, 0.06446188966228954)]

#### เรียงค่าความใกล้เคียง ที่ใกล้ index 1359 มากที่สุด

In [35]:
sorted(list(enumerate(cosin_sim[1359])) , reverse=True , key=lambda x : x[1])[:10]

[(1359, 1.0),
 (210, 0.3891142472988474),
 (428, 0.37978185033038503),
 (299, 0.20297937157029983),
 (1296, 0.20277087821870754),
 (2433, 0.1610673032949134),
 (65, 0.14794254761525155),
 (14, 0.14023408861343156),
 (10, 0.1328129006583444),
 (119, 0.12551519773218833)]

#### 3 ลำดับที่ค่าใกล้เคียง Batman มากที่สุด(ไม่นับตัวมันเอง) คือ index 210 , 428 , 299 

In [36]:
print('index 210 = ' , movie['title'][210])

index 210 =  Batman & Robin


In [37]:
print('index 428 = ' , movie['title'][428])

index 428 =  Batman Returns


In [38]:
print('index 299 = ' , movie['title'][299])

index 299 =  Batman Forever


#### Content-Based Filtering โดย movie_info

In [46]:
def recommend_movie_info(movie_name : str , top : int):
    count_metrix = tv.fit_transform(movie['movie_info'])
    cosin_sim = cosine_similarity(count_metrix , count_metrix)
    
    movie_index = movie[movie['title'] == movie_name].index.values[0]
    similar_movie = list(enumerate(cosin_sim[movie_index]))
    
    sort_similar_movie = sorted(similar_movie , key=lambda x : x[1] , reverse=True)
    
    no = 1
    for i , s in sort_similar_movie[1:top]:
        print(f"{no}. {movie['title'][i]}")
        no += 1

#### Content-Based Filtering โดย overview

In [47]:
def recommend_overview(movie_name : str , top : int):
    count_metrix = tv.fit_transform(movie['overview'])
    cosin_sim = cosine_similarity(count_metrix , count_metrix)
    
    movie_index = movie[movie['title'] == movie_name].index.values[0]
    similar_movie = list(enumerate(cosin_sim[movie_index]))
    
    sort_similar_movie = sorted(similar_movie , key=lambda x : x[1] , reverse=True)
    
    no = 1
    for i , s in sort_similar_movie[1:top]:
        print(f"{no}. {movie['title'][i]}")
        no += 1

#### เปรียบเทียบกันระหว่าง Content-Based Filtering โดย movie_info และ Content-Based Filtering โดย overview

In [48]:
recommend_movie_info('Batman' , 11)

1. Batman & Robin
2. Batman Returns
3. Batman Forever
4. Superman III
5. Superman IV: The Quest for Peace
6. The Dark Knight
7. Man of Steel
8. Superman Returns
9. Batman Begins
10. Undercover Brother


In [49]:
recommend_overview('Batman' , 11)

1. The Dark Knight Rises
2. Batman Begins
3. The Dark Knight
4. Batman Returns
5. Batman & Robin
6. Batman Forever
7. Batman v Superman: Dawn of Justice
8. Wicked Blood
9. George and the Dragon
10. Chairman of the Board


#### Content-Based Filtering โดย movie_info มี Batman ภาคต่างๆ 5 เรื่อง จาก 10 เรื่องที่ใกล้เคียงมากที่สุด
#### Content-Based Filtering โดย overview มี Batman ภาคต่างๆ 7 เรื่อง จาก 10 เรื่องที่ใกล้เคียงมากที่สุด

### แล้วถ้านำ movie_info กับ overview มารวมเข้าด้วยกันล่ะ ?

In [43]:
movie['movie_info_overview'] = movie['movie_info'] + movie['overview']

#### function Content-Based Filtering โดย movie_info_overview

In [50]:
def recommend_movie_info_overview(movie_name : str , top : int):
    count_metrix = tv.fit_transform(movie['movie_info_overview'])
    cosin_sim = cosine_similarity(count_metrix , count_metrix)
    
    movie_index = movie[movie['title'] == movie_name].index.values[0]
    similar_movie = list(enumerate(cosin_sim[movie_index]))
    
    sort_similar_movie = sorted(similar_movie , key=lambda x : x[1] , reverse=True)
    
    no = 1
    for i , s in sort_similar_movie[1:top]:
        print(f"{no}. {movie['title'][i]}")
        no += 1

In [51]:
recommend_movie_info_overview('Batman' , 11)

1. Batman & Robin
2. Batman Returns
3. Batman Forever
4. The Dark Knight
5. The Dark Knight Rises
6. Batman Begins
7. Batman: The Dark Knight Returns, Part 2
8. Batman
9. Superman IV: The Quest for Peace
10. Man of Steel


#### Content-Based Filtering โดย movie_info_overview มี Batman ภาคต่างๆ 8 เรื่อง จาก 10 เรื่องที่ใกล้เคียงมากที่สุด